## Prepare data reader and model execution context

In [1]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.modelspec import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "account_abstraction"


# Select a model.
data_spec = ModelsDataSpec(root_path_prefix="blockbatch", models=[model_name])

# Select a block batch.
blockbatch_request = BlockBatchRequest.build(
    chains=["base"],
    range_spec="19911446:+1",
    root_paths_to_read=data_spec.input_root_paths,
)

# Construct readers
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)

# Show details for the batch we are processing.
pprint(readers[0])

# Ensure existence of data needed by the reader.
assert readers[0].inputs_ready

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)


2025-03-03 11:26:27 [debug    ] connecting to GOLDSKY Clickhouse client... filename=client.py lineno=56 process=96345
2025-03-03 11:26:27 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=96345
2025-03-03 11:26:27 [debug    ] loaded vault: 19 items         filename=vault.py lineno=79 process=96345
2025-03-03 11:26:27 [debug    ] initialized GOLDSKY Clickhouse client. filename=client.py lineno=61 process=96345
2025-03-03 11:26:28 [info     ] querying markers at time range min: 2024-09-17 22:30:39, max: 2024-09-17 22:30:39 root_paths=['blockbatch/account_abstraction_prefilter/entrypoint_logs_v1', 'blockbatch/account_abstraction_prefilter/entrypoint_traces_v1'] filename=request.py lineno=146 process=96345
2025-03-03 11:26:28 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=56 process=96345
2025-03-03 11:26:28 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=61 process=96345
2025-03-03 11:26:28 [info    

In [2]:
from op_analytics.datapipeline.models.code.account_abstraction.decoders import (
    register_4337_decoders,
)

register_4337_decoders(ctx)

from op_analytics.datapipeline.models.code.account_abstraction.abis import (
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0,
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0,
)


# Decoded UserOperationEvent logs.
user_ops = auxiliary_templates["account_abstraction/useroperationevent_logs"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": input_datasets[
            "blockbatch/account_abstraction_prefilter/entrypoint_logs_v1"
        ].as_subquery(),
    },
)

# Traces initiated on behalf of the UserOperationEvent sender
entrypoint_traces = auxiliary_templates[
    "account_abstraction/enriched_entrypoint_traces"
].create_table(
    duckdb_context=ctx,
    template_parameters={
        "prefiltered_traces": input_datasets[
            "blockbatch/account_abstraction_prefilter/entrypoint_traces_v1"
        ].as_subquery(),
        "uops": user_ops,
        "method_id_v6": INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0,
        "method_id_v7": INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0,
    },
)

# Data Quality Checks
errors = []
for name, val in auxiliary_templates.items():
    if "data_quality_check" in name:
        errors.extend(val.run_as_data_quality_check(duckdb_context=ctx))
if errors:
    raise Exception("\n\n".join([name] + [str(_) for _ in errors]))
else:
    print("Data Quality OK")


2025-03-03 11:26:28 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=283 process=96345
2025-03-03 11:26:28 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=96345 template=account_abstraction/useroperationevent_logs
2025-03-03 11:26:30 [info     ] duck db size: 12.3KB           filename=client.py lineno=40 process=96345
2025-03-03 11:26:30 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=283 process=96345
2025-03-03 11:26:30 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=96345 template=account_abstraction/enriched_entrypoint_traces
2025-03-03 11:26:40 [info     ] duck db size: 356.0MB          filename=client.py lineno=40 process=96345
2025-03-03 11:26:40 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=96345 template=account_abstraction/data_quality_check_01
2025-03-03 11:26:40 [info     ] Rendering quer

In [3]:
ctx.client.sql("SHOW TABLES")

┌─────────────────────────────────────────────────┐
│                      name                       │
│                     varchar                     │
├─────────────────────────────────────────────────┤
│ account_abstraction__enriched_entrypoint_traces │
│ account_abstraction__useroperationevent_logs    │
└─────────────────────────────────────────────────┘

In [6]:
## NON-STANDARD AA BUNDLE

ctx.client.sql(f"""
    SELECT 
        transaction_hash, 
        tx_from_address,
        bundler_address,
        trace_root, 
        trace_address, 
        innerhandleop_parent_trace_address,
        innerhandleop_trace_address, 
        is_from_sender, 
        is_innerhandleop,
        is_innerhandleop_subtrace,
        useropevent_success,
        status,
        error,
        userop_sender,
        from_address, 
        to_address,
        input
    FROM {entrypoint_traces} 
    WHERE transaction_hash = '0x6e7b82e957641c9178e0aa7d84d9ca170898d984928e271c714b2758ae57e461'
    ORDER BY TRY_CAST(trace_root AS INT), trace_address
""").show(max_rows=100)


┌────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────┬───────────────┬─────────────────────────────┬────────────────┬──────────────────┬───────────────────────────┬─────────────────────┬────────┬─────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┐
│                          transaction_hash                          │              tx_from_address               │                from_address                │                 to_address                 │ trace_root │ trace_address │ innerhandleop_trace_address │ is_from_sender │ is_innerhandleop │ is_innerhandleop_subtrace │ useropevent_success │ status │  error  │               userop_sender                │                from_address                │                 to_address         

In [5]:
ctx.client.sql(f"""
    SELECT * FROM {user_ops} 
    WHERE transaction_hash = '0x6e7b82e957641c9178e0aa7d84d9ca170898d984928e271c714b2758ae57e461'
    ORDER BY log_index
    """)

┌────────────┬─────────┬──────────┬─────────┬─────────────────┬──────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬───────────┬────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬──────────────────────────┬─────────┬───────────────┬───────────────┐
│     dt     │  chain  │ chain_id │ network │ block_timestamp │ block_number │                             block_hash                             │                          transaction_hash                          │ transaction_index │ log_index │              contract_address              │                             userophash                             │                   sender                   │                 paymaster                  │          nonce           